In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [9]:
headers={
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding':'gzip',
    'Accept-Language':'en-US,en;q=0.9,te;q=0.8',
    'Cache-Control':'no-cache',
    'Cookie':'_ga=GA1.3.1865816974.1657809308; preferences=%7B%22time_zone%22%3A%22America%2FNew_York%22%2C%22titles%22%3A%22romaji%22%7D; lang=%5B%22en-us%22%2C%22en%22%2C%22te%22%5D; cf_clearance=ciqXyh4RrB5k2P46hWN77AAfvFXTaHWoBYLYrAxU2l8-1688253416-0-150; _gid=GA1.3.133754087.1697915088; __Host-livechart_session=lI3nAwfvTq%2BXLhtlg0ZnSy6YPGiBldr%2ByqLX2LYawSfqlRcqig4Os1aX%2BUf7BCxsxyOxEBDxbxiacf8CYV%2FF4ZZoMTugAgHmXT0jh2VWrx5K0wWLppxMEnydmes4rCidwUOJxp%2FsxeoRhoqtWf%2BgZdWejpB6%2Fbo7B4GK4MIN%2Bs4%2BA6WbvpQETtRhahsrVsHjkVdoRBiiSAUBD48GT4ZtHKd0yIvLZLnSVm7VvjC1CiLzzacLUvn6%2BRTuLIKnOpqRppandvpy5%2BfKPgsM3RBXWpGsuPp2RISd2N986eYxnaujZhBRuu31ZLs%2FiqAeaYfeYClmZ4Vr2JKir5pS0YmwHvY2TOgcuL25Gl681dFwANYG2CHKhCMH0RnpSEdyh1%2F5uwlwdoPkwvam3f3HpsVPqANc7HM%3D--RHYtmLMzYquLIrP6--fHoFJ6AZALzMJv%2FSJzHP%2Fw%3D%3D; _ga_B6ETE2XHBJ=GS1.3.1697949028.67.1.1697949053.0.0.0',
    'Pragma':'no-cache',
    'Sec-Ch-Ua-Mobile':'?0',
    'Sec-Ch-Ua-Platform':'"macOS"',
    'Sec-Fetch-Dest':'document',
    'Sec-Fetch-Mode':'navigate',
    'Sec-Fetch-Site':'none',
    'Sec-Fetch-User':'?1',
    'Upgrade-Insecure-Requests':'1',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
}

In [10]:
URL = 'https://www.livechart.me/summer-2023/tv'
res = requests.get(URL,headers=headers)
res

<Response [200]>

In [80]:
soup = BeautifulSoup(res.text, "html.parser")
animes = soup.body.main.find_all('article',class_='anime', limit=25)
library_entities = soup.body.find_all('script', id='library_entries')

In [22]:
import json

anime_interests = json.loads(library_entities[0].text)
anime_interests.keys()

dict_keys(['11115', '12225', '11083', '11461', '11467', '11511', '321', '10448', '11207', '11532', '11900', '11164', '11170', '11921', '10939'])

In [19]:
print(animes[0].prettify())

<article class="anime" data-action="anime-card-list:search@window-&gt;anime-card#applySearch" data-alternate='["[ZH-HANS] 咒术回战 第二季","Sorcery Fight","JJK","咒术回战","주술회전","Guerra de Hechiceros","جوجوتسو كايسن","มหาเวทย์ผนึกมาร","جوجوتسو کایسن","Chú thuật hồi chiến","МАГИЧЕСКАЯ БИТВА","Batalhas de Feiticeiros","呪術廻戦 第2期","呪術廻戦 第2期「懐玉・玉折」","呪術廻戦 第2期「渋谷事変」","JUJUTSU KAISEN 2nd Season","JUJUTSU KAISEN Season 2: Hidden Inventory/Premature Death","JUJUTSU KAISEN Season 2: Hidden Inventory, Premature Death","JUJUTSU KAISEN Temporada 2: El tesoro escondido/El tesoro perdido","JUJUTSU KAISEN Temporada 2: El incidente de Shibuya","JUJUTSU KAISEN Temporada 2: Joya oculta","JUJUTSU KAISEN Temporada 2: Joya quebrada","JUJUTSU KAISEN: Hidden Inventory","JUJUTSU KAISEN: Premature Death","JUJUTSU KAISEN 2nd Season: Kaigyoku, Gyokusetsu","JUJUTSU KAISEN 2nd Season: Shibuya Jihen","JUJUTSU KAISEN 2nd Season: Shibuya Incident Arc"]' data-anime-card-list-target="card" data-anime-id="11083" data-controller="a

In [82]:
from collections import defaultdict
from pathlib import Path
from datetime import datetime

anime_data = defaultdict(dict)

path = Path('live_chart_errors.log')
def record_error(title, command, value, error):
    with path.open('a') as file:
        file.write(f"[{datetime.now()}] : error with finding the rating for anime('{title}')\n")
        file.write(f"Command : {command}\n")
        file.write(f"Result : {value}\n")
        file.write(f"Error : {e}\n\n")

# looping through animes on the website and creating a key-value pair in the anime_data
for anime in animes:
    try:
        id = int(anime['data-anime-id'])

        title = anime.h3.a.text
        anime_data[str(id)]['title'] = title

        # status = anime['data-library-status']

        anime_data[str(id)]['tags'] = [tag.text for tag in anime.ol.find_all('li')]

        poster = anime.find('div',class_='poster-container')
        recent_ep = int(poster.time['data-label'][2:]) if poster.time else None
        timestamp = poster.time['data-timestamp'] if poster.time else None
        next_ep = datetime.fromtimestamp(int(timestamp)).strftime("%Y-%m-%d %H:%M:%S") if timestamp else None
        image = poster.img['src']
        try:
            rating = float(poster.find('div',class_='anime-extras').text)
        except Exception as e:
            record_error(title, "poster.find('div',class_='anime-extras')", poster.find('div',class_='anime-extras'), e)
            rating = 0

        info = anime.find('div',class_='anime-info')

        studios = [studio.text for studio in info.ul.find_all('li')]
        anime_source = info.find('div',class_='anime-source')
        anime_eps = info.find('div',class_='anime-episodes').text.strip()
        # print(title, anime_eps)
        if '×' in anime_eps:
            number_eps = anime_eps.split(' × ')[0].split(' ')[0]
            number_eps = int(number_eps) if '?' not in number_eps else 0
            aime_duration = int(anime_eps.split(' × ')[1][:-1])
        else:
            number_eps = 1
            aime_duration = int(anime_eps[:-1])

        summary = '\n'.join([p.text for p in info
                            .find('div',class_='anime-synopsis')
                            .find_all('p',class_=None)])
        try:
            info_source = info.find('div',class_='anime-synopsis')\
                                .find('p',class_='text-italic').text.split(' ')[1][:-1]
        except Exception as e:
            record_error(title, "info.find('div',class_='anime-synopsis')\
                                .find('p',class_='text-italic').text.split(' ')[1][:-1]", \
                                info.find('div',class_='anime-synopsis')\
                                .find('p',class_='text-italic').text.split(' ')[1][:-1],
                                e)
            info_source = None

    except Exception as e:
        print(title)
        print(e)
